<a href="https://colab.research.google.com/github/Datbwoyyy/Egbo-Victor/blob/Datbwoyyy-patch-1/Cifar_10_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'cifar10-image-recognition:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F366471%2F714968%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240924%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240924T164229Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dd56b50adcd8885dbb18e46591ab3d29c3749ccb54c800516cac71a64a2993fda3ce6e50cbec8be2b1c1140a2b23e4446ee655bed88dec18ec1f254fa343f7f90d52c446dc271e1a6be277e8a3fcbd475cb79de901135f775af2f0b673f7a73f695eec8071640ddd59602f414ccd8f9d8df660421bd0b30a37b2ef2caa890a521cab01381a1309f1970cae878669a639dd98c34b89422f765ad7f35928f7e8b1f4637b4410196fe9cd5579153a71a4086732e87777b1367957cc5919d220fdc9c74a9072c50f7a60673871d8128d527bc778326fbd820b09bfb09ca1c241396749401a53c9d533c8684f716eef84182a2939f68b6505c2440dfd5c376906834fe'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 1007971063 bytes downloaded
Downloaded and uncompressed: cifar10-image-recognition
Data source import complete.


## Data Preparation and Augmentation
* Getting the data ready including data exploration and cleaning.

* Augmentation to increase degree of randomness

In [17]:
# Importing libraries and dependancies
import  torch
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split, TensorDataset


# Load the NumPy array
train_data = np.load('/kaggle/input/cifar10-image-recognition/train.npy')
test_data = np.load('/kaggle/input/cifar10-image-recognition/test.npy')

# Assuming the NumPy array has shape (num_samples, 32, 32, 3) and label array named train_labels
train_labels = np.load('/kaggle/input/cifar10-image-recognition/train.npy')
test_labels = np.load('/kaggle/input/cifar10-image-recognition/test.npy')

# Create a TensorDataset
train_dataset = TensorDataset(torch.from_numpy(train_data), torch.from_numpy(train_labels))
test_dataset = TensorDataset(torch.from_numpy(train_data),torch.from_numpy(test_labels))# Create a TensorDataset
train_dataset = TensorDataset(torch.from_numpy(train_data), torch.from_numpy(train_labels))
test_dataset = TensorDataset(torch.from_numpy(train_data),torch.from_numpy(test_labels))

# Convert the image data to float32 before creating the TensorDataset
train_data = np.load('/kaggle/input/cifar10-image-recognition/train.npy').astype(np.float32)
train_labels = np.load('/kaggle/input/cifar10-image-recognition/train.npy')
test_data = np.load('/kaggle/input/cifar10-image-recognition/test.npy').astype(np.float32)
test_labels = np.load('/kaggle/input/cifar10-image-recognition/test.npy')

# Define data augmentation for training data
transform_train = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),# Flip horizontally
    transforms.RandomRotation(10), # Rotate the image slightly
    transforms.ColorJitter(brightness=0.2, contrast=0.2,saturation=0.2),# Random color jitter
    transforms.Resize((32,32)), # Ensuring images are 32 x 32
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)) # Normalize dataset
    ])

# Use basic transformations for the test data(no augmentation)
transform_test= transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
    ])
# Transforming the dataset using DataLoader
class TensorDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Create a TensorDataset
train_dataset = TensorDataset(torch.from_numpy(train_data), torch.from_numpy(train_labels))
test_dataset = TensorDataset(torch.from_numpy(train_data),torch.from_numpy(test_labels))

In [18]:
# sPLIT train data into training and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset,[train_size,val_size])

In [19]:
# DataLoaders
train_loader = DataLoader(train_dataset,batch_size=64,shuffle=True)
val_Loader = DataLoader(val_dataset,batch_size=64,shuffle=False)



## Architecture of the Model
This Model will involve a more advanced CNN Architecture with additional layers,dropout, and batch normalization to improve accuracy.

In [20]:
import torch.nn as nn
import torch.nn.functional as F

class AugmentedCIFARCNN(nn.Module):
    def __init__(self):
        super(AugmentedCIFARCNN, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)

        # Max pooling
        self.pool = nn.MaxPool2d(2, 2)

        # Fully connected layers
        self.fc1 = nn.Linear(256 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)

        # Dropout
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        print("Input shape:", x.shape)
        # First convolution block
        x = self.pool(F.relu(self.bn1(self.conv1(x))))

        # Second convolution block
        x = self.pool(F.relu(self.bn2(self.conv2(x))))

        # Third convolution block
        x = self.pool(F.relu(self.bn3(self.conv3(x))))

        # Flatten the output from convolution layers
        x = x.view(-1, 256 * 4 * 4)

        # Fully connected layers
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Initialize the model
model = AugmentedCIFARCNN()


In [34]:
import torch
import numpy as np
from torch import nn
import torch.nn.functional as F

class AugmententedCIFARCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        # Print the input shape
        print("Input shape:", x.shape)

        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        # Print the shape before flattening
        print("Shape before view:", x.shape)

        # Flatten the output from convolution layers
        x = x.view(-1, 64 * 4 * 4) # Modified to match the convolutional layers output

        # Print the shape after flattening
        print("Shape after view:", x.shape)

        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = AugmententedCIFARCNN()


Train data shape: torch.Size([1, 3, 32, 32])
Test data shape: torch.Size([1, 3, 32, 32])
Input shape: torch.Size([25, 32, 3])
Labels shape: torch.Size([25, 32, 3])
Input shape: torch.Size([25, 32, 3])


RuntimeError: Given groups=1, weight of size [16, 3, 3, 3], expected input[1, 25, 32, 3] to have 3 channels, but got 25 channels instead

## Training  Optimization
We will accomplish this using
* Learning rate scheduling
* Early stopping
* Adam Optimizer

In [21]:
import urllib.request

url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
# Or a more general website URL

try:
    response = urllib.request.urlopen(url)
    print("URL is reachable")
except urllib.error.URLError as e:
    print("URL is not reachable:", e)

URL is reachable


In [38]:
import torch
import numpy as np
from torch import nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transformations (e.g., normalization)
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load CIFAR-10 training data
trainset = datasets.CIFAR10(root= './data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

# Removed the lines that added a dimension for batch size
# train_data = np.expand_dims(train_data, axis=0)  # Remove this line
# test_data = np.expand_dims(test_data, axis=0)  # Remove this line

#print("Train data shape:", train_data.shape)  # Expected shape: (num_samples, 32, 32, 3)
#print("Test data shape:", test_data.shape)

# Check if train_data is already a tensor
#if not isinstance(train_data, torch.Tensor):
#    train_data = torch.from_numpy(train_data).float().permute(0,3,1,2) # Convert train_data to a PyTorch tensor first

# Check if test_data is already a tensor
#if not isinstance(test_data, torch.Tensor):
#    test_data = torch.from_numpy(test_data).float().permute(0,3,1,2)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 10
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        # Ensure inputs are float32 and move to the correct device (CPU/GPU)
        inputs = inputs.float()
        # Check the shape of the inputs and labels
        #inputs = inputs.permute(2, 0, 1).unsqueeze(0)
        #labels = labels.permute(2, 0, 1).unsqueeze(0)
        print("Input shape:", inputs.shape)
        print("Labels shape:", labels.shape)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        #The ouput of the model and labels need to have the same shape
        print("Output shape:", outputs.shape)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:  # Print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

Streaming output truncated to the last 5000 lines.
Input shape: torch.Size([4, 3, 32, 32])
Labels shape: torch.Size([4])
Input shape: torch.Size([4, 3, 32, 32])
Shape before view: torch.Size([4, 64, 4, 4])
Shape after view: torch.Size([4, 1024])
Output shape: torch.Size([4, 10])
Input shape: torch.Size([4, 3, 32, 32])
Labels shape: torch.Size([4])
Input shape: torch.Size([4, 3, 32, 32])
Shape before view: torch.Size([4, 64, 4, 4])
Shape after view: torch.Size([4, 1024])
Output shape: torch.Size([4, 10])
Input shape: torch.Size([4, 3, 32, 32])
Labels shape: torch.Size([4])
Input shape: torch.Size([4, 3, 32, 32])
Shape before view: torch.Size([4, 64, 4, 4])
Shape after view: torch.Size([4, 1024])
Output shape: torch.Size([4, 10])
Input shape: torch.Size([4, 3, 32, 32])
Labels shape: torch.Size([4])
Input shape: torch.Size([4, 3, 32, 32])
Shape before view: torch.Size([4, 64, 4, 4])
Shape after view: torch.Size([4, 1024])
Output shape: torch.Size([4, 10])
Input shape: torch.Size([4, 3, 32

## Prediction And Submission
We will carry out predictions for the test set and then submit


In [41]:
import pandas as pd
import torch
# Load the best model for inference
# Check if the model file exists
try:
    model.load_state_dict(torch.load('best_model.pth'))
except FileNotFoundError:
    print("Error: 'best_model.pth' not found. Please ensure the model file exists and is in the correct directory.")
    # Handle the error appropriately, e.g., exit or use a default model
    #exit()

# Put the model in evaluation mode
model.eval()

# list to hold predictions
predictions = []

#Define the testloader
!pip install torchvision
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

# Disable gradient calculations for inference
with torch.no_grad():
  for data in testloader: # Make sure you are using the correct variable name
    inputs, labels = data # Assuming testloader contains labels as well
    outputs = model(inputs)
    _, predicted = torch.max(outputs,1)
    predictions.extend(predicted.cpu().numpy())

# Assuming class_names is defined elsewhere
class_names = trainset.classes

# Prepare the submission dataframe
submission_df = pd.DataFrame({
    'id': list(range(1, len(predictions)+1)),
    'label': [class_names[p] for p in predictions]
})

# save the submission file
submission_df.to_csv('submission.csv',index= False)

print("Submission file saved as 'submission.csv'")

<ipython-input-41-bf2b154ae82f>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


Streaming output truncated to the last 5000 lines.
Shape after view: torch.Size([4, 1024])
Input shape: torch.Size([4, 3, 32, 32])
Shape before view: torch.Size([4, 64, 4, 4])
Shape after view: torch.Size([4, 1024])
Input shape: torch.Size([4, 3, 32, 32])
Shape before view: torch.Size([4, 64, 4, 4])
Shape after view: torch.Size([4, 1024])
Input shape: torch.Size([4, 3, 32, 32])
Shape before view: torch.Size([4, 64, 4, 4])
Shape after view: torch.Size([4, 1024])
Input shape: torch.Size([4, 3, 32, 32])
Shape before view: torch.Size([4, 64, 4, 4])
Shape after view: torch.Size([4, 1024])
Input shape: torch.Size([4, 3, 32, 32])
Shape before view: torch.Size([4, 64, 4, 4])
Shape after view: torch.Size([4, 1024])
Input shape: torch.Size([4, 3, 32, 32])
Shape before view: torch.Size([4, 64, 4, 4])
Shape after view: torch.Size([4, 1024])
Input shape: torch.Size([4, 3, 32, 32])
Shape before view: torch.Size([4, 64, 4, 4])
Shape after view: torch.Size([4, 1024])
Input shape: torch.Size([4, 3, 32,